<a href="https://colab.research.google.com/github/Kadge-Jatin/Igest-Development/blob/main/model_v2/Conversion_into_TFLite_'model_v2'.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This code converts 'model_v2' into TFlite model 'model_v2.tflte'and then into .h file**<br>
Author: Jatin Kadge<br>
Roll No.: EE23D021

In [ ]:
%tensorflow_version 2.1

#%tensorflow_version` only switches the major version: 1.x or 2.x.
#You set: `2.1`. This will be interpreted as: `2.x`.

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
from tensorflow.keras import layers

In [ ]:
# Print versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
#print('keras ' + tf.keras.version.VERSION)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.23.5
TensorFlow 2.13.0


In [ ]:
tflite_model_name = 'tflite_model_v2'  # Will be given .tflite suffix
c_model_name = 'c_model_v2'       # Will be given .h suffix

In [ ]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
  #Load the Saved model
  loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/Wearable_devices_newdata/ML_models/ML models - Final/model_v2.keras")

In [ ]:
# View model
loaded_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_20 (Conv1D)          (None, 8, 64)             1216      
                                                                 
 conv1d_21 (Conv1D)          (None, 6, 64)             12352     
                                                                 
 dropout_10 (Dropout)        (None, 6, 64)             0         
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 3, 64)             0         
 ng1D)                                                           
                                                                 
 flatten_10 (Flatten)        (None, 192)               0         
                                                                 
 dense_20 (Dense)            (None, 100)               19300     
                                                     

In [ ]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

import os
basic_model_size = os.path.getsize("tflite_model_v2.tflite")
print("Model is %d bytes" % basic_model_size)

Model is 42232 bytes


In [ ]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [ ]:
# Write TFLite model to a C source (or header) file
with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, c_model_name))